In [1]:
# SecureString parameter stored in AWS System Manager that holds a GitHub personal access token.
parameter_name = 'github_token'

# Repositories for which reports will be generated.
repo_names = ['aws-amplify/amplify-android',
              'aws-amplify/aws-sdk-android',
              'awslabs/aws-mobile-appsync-sdk-android',
              'aws-amplify/amplify-ios',
              'aws-amplify/aws-sdk-ios']

In [2]:
# Retrieve the GitHub token from SSM to prevent oops-I-pushed-credentials-to-GitHub uh-ohs.

import boto3
from github import Github

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name=parameter_name, WithDecryption=True)
token = response['Parameter']['Value']
github = Github(token)

In [3]:
# Loop through each repository, grab all issues, and create a DataFrame for each.

import pandas as pd

repos = {}

for repo_name in repo_names:
    repo = github.get_repo(repo_name)
    issues = []

    for issue in repo.get_issues(state='all'):
        labels = pd.array([label.name for label in issue.labels])
        
        if issue.pull_request is None:
            issues.append([labels, issue.created_at, issue.closed_at])
        
    repos[repo_name] = pd.DataFrame(issues, columns=['labels', 'created_at', 'closed_at'])

In [4]:
# Using the DataFrames, process and grab the counts of issues. Go through each label and gets counts for each label and other applied labels.

from datetime import datetime
from IPython.display import display, HTML

seven_days_ago = datetime.now() - pd.Timedelta('7 days')
thirty_days_ago = datetime.now() - pd.Timedelta('30 days')

open_counts = []
label_counts = []

for repo_name in repo_names:
    r = repos[repo_name]
    labels = {label for labels in repos[repo_name].labels for label in labels}

    open_issues = r[~(r.closed_at > '1970-01-01')]
    last_week_open_issues = r[(r.created_at < seven_days_ago) & ~(r.closed_at < seven_days_ago)]
    last_month_open_issues = r[(r.created_at < thirty_days_ago) & ~(r.closed_at < thirty_days_ago)]
    open_issues_count = len(open_issues.index)
    last_week_open_issues_count = len(last_week_open_issues.index)
    last_month_open_issues_count = len(last_month_open_issues.index)
    
    open_counts.append([repo_name,
                        open_issues_count,
                        last_week_open_issues_count,
                        open_issues_count - last_week_open_issues_count,
                        last_month_open_issues_count,
                        open_issues_count - last_month_open_issues_count])
    
    for label in sorted(labels):
        label_mask = open_issues.labels.apply(lambda l: label in l)
        issues = open_issues[label_mask]
        last_week_label_issues = issues[(issues.created_at < seven_days_ago) & ~(issues.closed_at < seven_days_ago)]
        last_month_label_issues = issues[(issues.created_at < thirty_days_ago) & ~(issues.closed_at < thirty_days_ago)]
        label_issues_count = len(issues.index)
        last_week_label_issues_count = len(last_week_label_issues.index)
        last_month_label_issues_count = len(last_month_label_issues.index)
        
        if label_issues_count or last_week_label_issues_count or last_month_label_issues_count:
            label_counts.append([repo_name,
                                 label,
                                 '',
                                 label_issues_count,
                                 last_week_label_issues_count,
                                 label_issues_count - last_week_label_issues_count,
                                 last_month_label_issues_count,
                                 label_issues_count - last_month_label_issues_count])

            for other_label in labels - {label}:
                other_label_mask = issues.labels.apply(lambda l: other_label in l)
                other_label_issues = issues[other_label_mask]

                if not other_label_issues.empty:
                    last_week_other_label_issues = other_label_issues[(other_label_issues.created_at < seven_days_ago) & ~(other_label_issues.closed_at < seven_days_ago)]
                    last_month_other_label_issues = other_label_issues[(other_label_issues.created_at < thirty_days_ago) & ~(other_label_issues.closed_at < thirty_days_ago)]
                    other_label_issues_count = len(other_label_issues.index)
                    last_week_other_label_issues_count = len(last_week_other_label_issues.index)
                    last_month_other_label_issues_count = len(last_month_other_label_issues.index)

                    if other_label_issues_count or last_week_other_label_issues_count or last_month_other_label_issues_count:
                        label_counts.append([repo_name,
                                             label,
                                             other_label,
                                             other_label_issues_count,
                                             last_week_other_label_issues_count,
                                             other_label_issues_count - last_week_other_label_issues_count,
                                             last_month_other_label_issues_count,
                                             other_label_issues_count - last_month_other_label_issues_count])
                
open_df = pd.DataFrame(open_counts, columns=['Repo', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])
labels_df = pd.DataFrame(label_counts, columns=['Repo', 'Label', 'Other Label', 'Open', 'Last Week', 'WoW', 'Last Month', 'MoM'])

In [5]:
# Build the report for display.

pd.set_option('display.max_rows', 500)

display(HTML('<h1>Open Source – Ops Report</h1>'))
display(HTML(f'<em>Generated on {datetime.now()}'))

display(HTML('<h2>Open Issues</h2>'))
display(open_df)

display(HTML('<h2>Open Issues by Label</h2>'))

for repo_name in repo_names:
    display(HTML(f'<h3>{repo_name}</h3>'))
    display(labels_df[labels_df.Repo == repo_name])

,Repo,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-android,32,28,4,24,8
1,aws-amplify/aws-sdk-android,144,145,-1,136,8
2,awslabs/aws-mobile-appsync-sdk-android,44,45,-1,45,-1
3,aws-amplify/amplify-ios,53,71,-18,67,-14
4,aws-amplify/aws-sdk-ios,173,166,7,184,-11


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
0,aws-amplify/amplify-android,API,,8,8,0,8,0
1,aws-amplify/amplify-android,API,Feature Request,2,2,0,2,0
2,aws-amplify/amplify-android,API,Improvement,1,1,0,1,0
3,aws-amplify/amplify-android,API,Bug,4,4,0,4,0
4,aws-amplify/amplify-android,API,Core,1,1,0,1,0
5,aws-amplify/amplify-android,API,Clarification Needed,1,1,0,1,0
6,aws-amplify/amplify-android,Analytics,,1,1,0,1,0
7,aws-amplify/amplify-android,Analytics,Improvement,1,1,0,1,0
8,aws-amplify/amplify-android,Auth,,1,1,0,0,1
9,aws-amplify/amplify-android,Auth,Improvement,1,1,0,0,1


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
49,aws-amplify/aws-sdk-android,APIGateway,,4,4,0,4,0
50,aws-amplify/aws-sdk-android,APIGateway,Usage Question,3,3,0,3,0
51,aws-amplify/aws-sdk-android,APIGateway,Feature Request,1,1,0,1,0
52,aws-amplify/aws-sdk-android,AWSMobileClient,,38,38,0,33,5
53,aws-amplify/aws-sdk-android,AWSMobileClient,Cognito,3,3,0,2,1
54,aws-amplify/aws-sdk-android,AWSMobileClient,Service,1,1,0,1,0
55,aws-amplify/aws-sdk-android,AWSMobileClient,Documentation,2,2,0,2,0
56,aws-amplify/aws-sdk-android,AWSMobileClient,Usage Question,9,9,0,7,2
57,aws-amplify/aws-sdk-android,AWSMobileClient,Feature Request,17,17,0,17,0
58,aws-amplify/aws-sdk-android,AWSMobileClient,closing-soon-if-no-response,1,1,0,0,1


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
198,awslabs/aws-mobile-appsync-sdk-android,AppSync,,41,41,0,40,1
199,awslabs/aws-mobile-appsync-sdk-android,AppSync,Feature Request,13,13,0,13,0
200,awslabs/aws-mobile-appsync-sdk-android,AppSync,Question,9,9,0,8,1
201,awslabs/aws-mobile-appsync-sdk-android,AppSync,Product Review,2,2,0,2,0
202,awslabs/aws-mobile-appsync-sdk-android,AppSync,Documentation,2,2,0,2,0
203,awslabs/aws-mobile-appsync-sdk-android,AppSync,Infrastructure,2,2,0,2,0
204,awslabs/aws-mobile-appsync-sdk-android,AppSync,Bug,16,16,0,16,0
205,awslabs/aws-mobile-appsync-sdk-android,AppSync,Requesting Feedback,2,2,0,2,0
206,awslabs/aws-mobile-appsync-sdk-android,AppSync,Pending,1,1,0,1,0
207,awslabs/aws-mobile-appsync-sdk-android,AppSync,Codegen,4,4,0,4,0


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
252,aws-amplify/amplify-ios,API,,8,8,0,6,2
253,aws-amplify/amplify-ios,API,enhancement,2,2,0,2,0
254,aws-amplify/amplify-ios,API,bug,2,2,0,2,0
255,aws-amplify/amplify-ios,API,DataStore,4,4,0,2,2
256,aws-amplify/amplify-ios,API,P0,2,2,0,2,0
257,aws-amplify/amplify-ios,Analytics,,1,1,0,1,0
258,aws-amplify/amplify-ios,Analytics,Requesting Feedback,1,1,0,1,0
259,aws-amplify/amplify-ios,Build,,3,3,0,2,1
260,aws-amplify/amplify-ios,Build,Core,1,1,0,1,0
261,aws-amplify/amplify-ios,Build,DataStore,1,1,0,0,1


,Repo,Label,Other Label,Open,Last Week,WoW,Last Month,MoM
316,aws-amplify/aws-sdk-ios,APIGateway,,7,6,1,6,1
317,aws-amplify/aws-sdk-ios,APIGateway,Cognito,1,0,1,0,1
318,aws-amplify/aws-sdk-ios,APIGateway,Question,1,1,0,1,0
319,aws-amplify/aws-sdk-ios,APIGateway,Feature Request,4,4,0,4,0
320,aws-amplify/aws-sdk-ios,APIGateway,Bug,1,1,0,1,0
321,aws-amplify/aws-sdk-ios,APIGateway,Investigating,1,1,0,1,0
322,aws-amplify/aws-sdk-ios,AWSMobileClient,,38,37,1,33,5
323,aws-amplify/aws-sdk-ios,AWSMobileClient,Cognito,10,10,0,7,3
324,aws-amplify/aws-sdk-ios,AWSMobileClient,Service,2,2,0,2,0
325,aws-amplify/aws-sdk-ios,AWSMobileClient,Requesting Feedback,4,4,0,4,0
